# Lab 2 - Correlation Power Analysis with ChipWhisperer

**SUMMARY:** *Last time, we saw how power anlaysis can be used to break a password check. In this lab, we'll use ChipWhisperer Analyzer to do a Correlation Power Analysis attack to recover an AES key.*

**LEARNING OUTCOMES:**

* Use ChipWhisperer Analyzer to perform a CPA attack
* Plot additional information about the attack

## Projects

We'll proceed fairly quickly through this lab - our goal here is just to showcase the power of a CPA attack. We'll go into more detail about the attack in a later lab.

This time, we'll be using the `simpleserial-aes` firmware, located at `chipwhisperer/hardware/victims/firmware/simpleserial-aes`. `simpleserial` is a protocol that we use to transfer data to and from most of NewAE's target firmware. Documentation can be found at https://chipwhisperer.readthedocs.io/en/latest/simpleserial.html. There's no need to worry about most of that documentation - ChipWhisperer will take care of the details for you.

This time, the firmware mostly does two things - first, it allows us to set a key for the AES implementation. Our goal is to recover this key. It also lets us send plaintext to the target, which the target will then encrypt and send back to us. Note that while we know the key, we don't use it in our attack at all - we mostly just use it to see how well the attack is working in real time.

As with before, this tutorial can be completed with both hardware and previously recorded traces.

## \#HARDWARE

Most of the hardware setup is the same as before, so we'll move quickly through this:

In [ ]:
# Set hardware settings
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_SAM4S'
CRYPTO_TARGET='TINYAES128C' 
SS_VER='SS_VER_2_1'

In [ ]:
# Connect to ChipWhisperer
%run "../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
# compile firmware
cd ../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 -j

In [ ]:
# program firmware onto target
cw.program_target(scope, prog, "../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM))

The first major difference here is that we'll be storing our info in a ChipWhisperer Project instead of a NumPy array. Projects are a convenient way to interface power traces with ChipWhisperer Analyzer

In [ ]:
proj = cw.create_project("Lab 4_3", overwrite=True)

Jupyter allows us to plot asynchronously, meaning we can plot traces as we capture them. ChipWhisperer includes an convenince class to make this easy to setup and do:

In [ ]:
splot = cw.StreamPlot()
splot.plot()

Unlike last time, we'll get all our power traces at one time. We can make use of lots of ChipWhisperer convenience functions:

* `ktp = cw.ktp.Basic()` Object to generate our plaintext/key.
* `key, text = ktp.next()` Generate the next plaintext (key stays constant)
* `cw.capture_trace(scope, target, text, key)` capture a power trace using plaintext/key

Again, we'll go into more detail about what we're sending/receiving, how the plaintext is being generated, etc. in the next lab.

In [ ]:
from tqdm.notebook import trange
import numpy as np
import time

ktp = cw.ktp.Basic() # default - fixed key, random plaintext

N = 50
for i in trange(N, desc='Capturing traces'):
    key, text = ktp.next() # new plaintext, same key
    trace = cw.capture_trace(scope, target, text, key) # set key, send plaintext, receive ciphertext, capture power trace
    if not trace:
        continue
    
    proj.traces.append(trace) # store our recorded trace in our project
    
    # Update our plot with a new trace
    splot.update(trace.wave) # wave is the name for the data for our power trace

In [ ]:
scope.dis()
target.dis()

## \#SIMULATED

If you don't have hardware, you can instead just open a project with the power traces already recorded in it.

In [ ]:
# uncomment to run
import chipwhisperer as cw
proj = cw.open_project("traces/Lab_4_3.cwp")

## ChipWhisperer Analyzer

To make side channel attacks easier, ChipWhisperer includes an analysis tool, called Analyzer, that can do most of the work for us.

We can access Analyzer via `chipwhisperer.analyzer`:

In [ ]:
import chipwhisperer.analyzer as cwa

Next, we'll set our leakage model. AES, like many algorithms, can be broken down into multiple smaller steps. The leakage model basically selects which of these steps to target. For this lab, we'll be using the `sbox_output` step, which is near the beginning of AES. Which model you need to use will depend on how AES is implemented, but for software AES, the SBox is generally a good place to target.

In [ ]:
leak_model = cwa.leakage_models.sbox_output

The rest of the setup only takes 1 line:

In [ ]:
attack = cwa.cpa(proj, leak_model)

If you want to see the attack settings, you can print the cpa object:

In [ ]:
print(attack)

Running the attack is also done in a single line:

In [ ]:
results = attack.run()

Let's see if we got the AES key:

In [ ]:
print(cw.bytearray(results.key_guess()))
print(proj.keys[0]) # key we recorded earlier

We can also get a nicer printout showing the correlation we got for our guesses:

In [ ]:
print(results)
print(proj.keys[0])

Analyzer also has the ability to report data as we run the attack:

In [ ]:
cb = cwa.get_jupyter_callback(attack)
results = attack.run(cb, 10) # report every time 10 traces are processed

You should see the correct key appear as red text. We're able to do this because we know the key. In a more realistic scenario, a large difference in correlation between the top guess and the next best guess is a good way to see if your attack worked.

## Plotting Additional Data

### Output vs. Time

Now that we've seen a CPA attack work, we can use ChipWhisperer Analyzer to get some additional insights into the attack. Note that this information is only available because we know the key and therefore wouldn't be available to an attacker until they recovered the key. As such, this is of more interest to someone evaluating different implementations of AES.

One thing that we often want to look at with a CPA attack is when in time the attack is succeeding at - there's only a short period that the data we care about is being manuipulated at. We can use ChipWhisperer's `output_vs_time()` function to plot this.

Begin by generating the data for additional plots:

In [ ]:
plot_data = cwa.analyzer_plots(results)

Then, all we need to do is call the `output_vs_time()` method and plot the data. This function will return four datasets - the first isn't too important, but the next three are. The first of these is a plot of the correlation of the correct key guess over time, while the next two are the max and minimum correlation over time of the rest of the guesses.

Plotting like this:

In [ ]:
plot = cw.plot()
data = plot_data.output_vs_time(0)
plot *= cw.plot(data[1], label=str(0)).options(color="green") \
* cw.plot(data[2], label=str(0)).options(color="black") \
* cw.plot(data[3], label=str(0)).options(color="black")
plot

should show two black plots and a green plot, with the green plot being the correct key. **Record the location of this spike by zooming in on the plot. We'll be using this value in the next lab.**

In [ ]:
import holoviews as hv
def byte_to_color(idx):
    return hv.Palette.colormaps['Paired'](idx/16)

plot = cw.plot()
colours = []
for i in range(0, 16):
    data = plot_data.output_vs_time(i)
    plot *= cw.plot(data[1], label=str(i)).options(color=byte_to_color(i)) \
    * cw.plot(data[2], label=str(i)).options(color="black") \
    * cw.plot(data[3], label=str(i)).options(color="black")
plot

You should be able to see 16 distinct spikes corresponding to each of the key bytes. As you can see, this implementation manipulates one byte at a time, but other AES implementations may manipulate multiple. For software implementations, 32-bit (4 byte) implementations are common. With hardware AES, a device may work on all 16-bytes at once, or even do multiple steps of AES in a single clock cycle!

### PGE vs. Traces

Another thing we may be interested in when evaluating AES implementations is how many traces it took to recover the key. While we only needed 50 traces in this lab, hardware implementations especially can require orders of magnitude more traces to recover a key. You can probably see the usefulness of knowing whether an attack took 10 000 traces to work vs. taking 1 000 000 traces to work, especially if the device you're looking at generates new keys periodically.

Once way we can look at this information is a PGE vs. Traces plot. PGE stands for Partial Guessing Entropy. While the name sounds complicated, it just represents how many incorrect key guesses have a higher correlation than a correct key byte. For example, if the key byte is `0x2b` and `0xad`, `0x24`, and `0x7e` all have higher correlations, then the PGE for that part of the key is 3.

We can get a PGE vs. Traces plot via `plot_data.pge_vs_trace()`:

In [ ]:
ret = plot_data.pge_vs_trace(0)
plot = cw.plot()
#curve = hv.Curve((ret[0],ret[1]), "Traces Used in Calculation", "Partial Guessing Entrop of Byte")
for bnum in range(0, 16):
    ret = plot_data.pge_vs_trace(bnum)
    plot *= cw.plot((ret[0], ret[1]))
plot

You should see a high PGE at the beginning, but as we process more traces, the PGE should drop.

### Correlation vs. Traces

Another interesting plot is Correlation vs. Traces. As you will see, all the subkey guesses start of with large correlations, but all of them except for the correct guess quickly drop off. If you didn't know the key, at what point would you be sure that the guess with the highest correlation was actually the correct subkey?

Since this plot actually has a lot of data in it, we'll just plot the data for one byte of the key this time:

In [ ]:
plot = cw.plot()
data = plot_data.corr_vs_trace(0)
for guess in data[1]:
    plot *= cw.plot((data[0], guess))
plot

## Conclusions & Next Steps

Congratulations on completing your first power analysis attack against AES!

As you've seen, Analyzer makes launching a CPA attack easy. It also has the advantage of capturing some interesting data for us, and reporting the attack success every so often.

Next lab, we'll take a more in depth look at how this attack works.

---
<small>NO-FUN DISCLAIMER: This material is Copyright (C) NewAE Technology Inc., 2015-2023. ChipWhisperer is a trademark of NewAE Technology Inc., claimed in all jurisdictions, and registered in at least the United States of America, European Union, and Peoples Republic of China.

Tutorials derived from our open-source work must be released under the associated open-source license, and notice of the source must be *clearly displayed*. Only original copyright holders may license or authorize other distribution - while NewAE Technology Inc. holds the copyright for many tutorials, the github repository includes community contributions which we cannot license under special terms and **must** be maintained as an open-source release. Please contact us for special permissions (where possible).

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.</small>